In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
# import ptitprince as pt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last"
import pingouin as pg
import statsmodels.api as sm
# import language_tool_python as ltp
import sys
sys.path.append('..')  
# from src.visualization.plot import raincloud, raincloud2, RainCloud
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

    

In [5]:
df = pd.read_csv('../data/processed/restored_dv2_dv3_dv2h_cleaned.csv')

# just print some descriptive info
for engine in df.engine.unique():
    print('Experiment=', end=' ')
    print(engine)
    print('-'*15)
    print('Size=', end=' ')
    print(df[df.engine==engine].shape)
    # print n subject
    print('N datasets =', end=' ')
    # group by engine and prolific_id, then count unique dataset_id
    dd = df[df.engine==engine].groupby(['prolific_id']).dataset_id.unique()
    n = sorted([i[0] for i in dd.values])
    print(len(n))
    print('N participants =', end=' ')
    print(df[df.engine==engine].dataset_id.nunique())

    # check if n unique dataset_id == n unique prolific_id
    if len(n) != df[df.engine==engine].dataset_id.nunique():
        print('WARNING: n unique dataset_id != n unique prolific_id')
        # show which dataset_id is doubled
        print('Doubled dataset_id=', end=' ')
        print([i for i in n if n.count(i) > 1])

    print('N trials = ', end='')
    # print n trials per subject
    print(df[df.engine==engine].prolific_id.value_counts().unique()[0])
    print()

df.loc[(((df.a0=='yes') & (df.agree1==1)) | ((df.a0=='no') & (df.agree1==0))), 'endorse'] = 1
df.loc[(((df.a0=='yes') & (df.agree1==0)) | ((df.a0=='no') & (df.agree1==1))), 'endorse'] = 0
df['endorse'] = df.endorse.astype(int)

Experiment= dv2
---------------
Size= (3542, 29)
N datasets = 77
N participants = 77
N trials = 46

Experiment= dv3
---------------
Size= (3420, 29)
N datasets = 76
N participants = 76
N trials = 45

Experiment= dv2_humanized
---------------
Size= (3542, 29)
N datasets = 77
N participants = 77
N trials = 46



In [6]:
avg_source = df.groupby(['engine', 'prolific_id', 'cond', 'source'], as_index=False).mean(numeric_only=True)
avg_answer = df.groupby(['engine', 'prolific_id', 'cond', 'answer'], as_index=False).mean(numeric_only=True)

count = 0
nan = 0
avg = None
bias = []

for id in avg_source.prolific_id.unique():
    for cond in avg_source.cond.unique():
        # get engine using id
        engine = avg_source[avg_source.prolific_id==id].engine.values[0]
        count += 1
        del avg
        avg = avg_source
        bias_source = avg[(avg.source=='Human') & (avg.prolific_id==id) & (avg.cond==cond)].agree.values[0]\
            - avg[(avg.source=='AI') & (avg.prolific_id==id) & (avg.cond==cond)].agree.values[0]
        # assert that there is only one value for each condition
        assert len(avg[(avg.source=='Human') & (avg.prolific_id==id) & (avg.cond==cond)].agree) == 1
        assert len(avg[(avg.source=='AI') & (avg.prolific_id==id) & (avg.cond==cond)].agree) == 1
        try: 
            del avg
            avg = avg_answer
            bias_answer = \
                avg[(avg.answer=='Human') & (avg.prolific_id==id) & (avg.cond==cond)].agree.values[0]\
                - avg[(avg.answer=='AI') & (avg.prolific_id==id) & (avg.cond==cond)].agree.values[0]
        except:
            nan += 1
            bias_answer = None 
        
        bias.append(
            {
                'engine': engine,
                'prolific_id': id,
                'cond': cond,
                'bias_source': bias_source,
                'bias_answer': bias_answer
            }
        )

bias = pd.DataFrame(bias)


In [4]:
bias2 = []
avg_source = df.groupby(['engine', 'prolific_id',  'source'], as_index=False).mean(numeric_only=True)
avg_answer = df.groupby(['engine', 'prolific_id',  'answer'], as_index=False).mean(numeric_only=True)
avg = None

for id in avg_source.prolific_id.unique():
        # get engine using id
    engine = avg_source[avg_source.prolific_id==id].engine.values[0]
    count += 1
    del avg
    avg = avg_source
    bias_source = avg[(avg.source=='Human') & (avg.prolific_id==id)].agree.values[0]\
        - avg[(avg.source=='AI') & (avg.prolific_id==id)].agree.values[0]
    # assert that there is only one value for each condition
    assert len(avg[(avg.source=='Human') & (avg.prolific_id==id)].agree) == 1
    assert len(avg[(avg.source=='AI') & (avg.prolific_id==id)].agree) == 1
    try: 
        del avg
        avg = avg_answer
        bias_answer = \
            avg[(avg.answer=='Human') & (avg.prolific_id==id)].agree.values[0]\
            - avg[(avg.answer=='AI') & (avg.prolific_id==id)].agree.values[0]
    except:
        nan += 1
        bias_answer = None 
    
    bias2.append(
        {
            'engine': engine,
            'prolific_id': id,
            'bias_source': bias_source,
            'bias_answer': bias_answer
        }
    )

bias2 = pd.DataFrame(bias2)

# Table 1


## 4.1 judgments


In [5]:
dfh = pd.read_csv('../data/processed/humans.csv')
df2 = pd.read_csv('../data/processed/dv2.csv')
df3 = pd.read_csv('../data/processed/dv3.csv')

In [6]:

pg.pairwise_ttests(data=dfh, dv='yes', within='cond', subject='prolific_id', padjust='bonf', effsize='cohen')

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,cond,impersonal_moral,non_moral,True,True,-2.320087,30.0,two-sided,0.027325,0.081974,bonf,1.925,-0.429355
1,cond,impersonal_moral,personal_moral,True,True,2.808337,30.0,two-sided,0.008674,0.026023,bonf,5.026,0.690784
2,cond,non_moral,personal_moral,True,True,4.930579,30.0,two-sided,0.000028,0.000085,bonf,810.225,1.105966


In [7]:

pg.pairwise_ttests(data=df2, dv='yes', within='cond', subject='iter', padjust='bonf', effsize='cohen').round(6)

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,cond,impersonal_moral,non_moral,True,True,-2.742182,29.0,two-sided,0.010348,0.031044,bonf,4.375,-0.672540
1,cond,impersonal_moral,personal_moral,True,True,10.051279,29.0,two-sided,0.000000,0.000000,bonf,1.657e+08,2.239542
2,cond,non_moral,personal_moral,True,True,9.636332,29.0,two-sided,0.000000,0.000000,bonf,6.738e+07,2.670451


In [8]:

pg.pairwise_ttests(data=df3, dv='yes', within='cond', subject='iter', padjust='bonf', effsize='cohen')

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,cond,impersonal_moral,non_moral,True,True,-42.764995,29.0,two-sided,9.425982e-28,2.827795e-27,bonf,2.183e+24,-11.633498
1,cond,impersonal_moral,personal_moral,True,True,49.282532,29.0,two-sided,1.626808e-29,4.880425e-29,bonf,1.095e+26,12.113217
2,cond,non_moral,personal_moral,True,True,112.807581,29.0,two-sided,6.919688e-40,2.075906e-39,bonf,1.12e+36,31.120950


## 4.2  Detection

In [9]:
avg = df.groupby(['engine', 'prolific_id'], as_index=False).mean(numeric_only=True)

d = []
for engine in avg.engine.unique():
    print(engine)
    table = pg.ttest(avg[avg.engine==engine].correct, 0.5).round(6)
    table['Exp.'] = engine
    d.append(table)
    
table = pd.concat(d)

table

dv2
dv2_humanized
dv3


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power,Exp.
T-test,10.494557,76,two-sided,0.0,"[0.62, 0.67]",1.195966,3.217e+13,1.0,dv2
T-test,8.247513,76,two-sided,0.0,"[0.56, 0.59]",0.939891,2.304e+09,1.0,dv2_humanized
T-test,13.885711,75,two-sided,0.0,"[0.68, 0.74]",1.592800,2.195e+19,1.0,dv3


In [10]:
# add a chance condition
avg = df[df.engine.isin(['dv2', 'dv3'])].groupby(['engine', 'prolific_id', 'cond'], as_index=False).mean(numeric_only=True)

to_add = []
for engine in avg.engine.unique():
    for id in avg[avg.engine==engine].prolific_id.unique():
        for cond in ['chance']:
            to_add.append(
                {
                    'engine': engine,
                    'prolific_id': id,
                    'cond': cond,
                    'correct': 0.5
                }
            )
to_add = pd.DataFrame(to_add)

avg = pd.concat([avg, to_add])

pg.pairwise_tests(dv='correct', within='cond', between="engine",
                   subject='prolific_id', data=avg, padjust='bonf', within_first=False, effsize='cohen').round(6)


,Contrast,engine,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,engine,-,dv2,dv3,False,True,-3.023768,149.94963,two-sided,0.002938,NaN,NaN,10.82,-0.489152
1,cond,-,chance,impersonal_moral,True,True,-14.641076,152.00000,two-sided,0.000000,0.000000,bonf,4.092e+27,-1.673949
2,cond,-,chance,non_moral,True,True,-14.153598,152.00000,two-sided,0.000000,0.000000,bonf,2.165e+26,-1.618215
3,cond,-,chance,personal_moral,True,True,-12.458568,152.00000,two-sided,0.000000,0.000000,bonf,7.261e+21,-1.424418
4,cond,-,impersonal_moral,non_moral,True,True,3.507105,152.00000,two-sided,0.000596,0.003575,bonf,30.066,0.329911
5,cond,-,impersonal_moral,personal_moral,True,True,0.215070,152.00000,two-sided,0.830001,1.000000,bonf,0.092,0.017622
6,cond,-,non_moral,personal_moral,True,True,-3.068214,152.00000,two-sided,0.002550,0.015302,bonf,8.0,-0.280546
7,engine * cond,dv2,chance,impersonal_moral,True,True,-8.898032,76.00000,two-sided,0.000000,0.000000,bonf,3.687e+10,-1.434048
8,engine * cond,dv2,chance,non_moral,True,True,-9.560496,76.00000,two-sided,0.000000,0.000000,bonf,6.198e+11,-1.540813
9,engine * cond,dv2,chance,personal_moral,True,True,-7.235882,76.00000,two-sided,0.000000,0.000000,bonf,3.204e+07,-1.166168


## 4.3 Alignment

In [11]:
# add a chance condition
avg = bias[bias.engine.isin(['dv2', 'dv3'])].groupby(['engine', 'prolific_id', 'cond'], as_index=False).mean(numeric_only=True)

to_add = []
for engine in avg.engine.unique():
    for id in avg[avg.engine==engine].prolific_id.unique():
        for cond in ['chance']:
            to_add.append(
                {
                    'engine': engine,
                    'prolific_id': id,
                    'cond': cond,
                    'bias_answer': 0,
                    'bias_source': 0
                }
            )
to_add = pd.DataFrame(to_add)

avg = pd.concat([avg, to_add])

pg.pairwise_tests(dv='bias_source', within='cond', between="engine",
                   subject='prolific_id', data=avg, padjust='bonf', within_first=False, effsize='cohen').round(6)

,Contrast,engine,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,engine,-,dv2,dv3,False,True,0.429879,144.823819,two-sided,0.667922,NaN,NaN,0.19,0.069597
1,cond,-,chance,impersonal_moral,True,True,-1.736090,152.000000,two-sided,0.084574,0.507445,bonf,0.39,-0.198491
2,cond,-,chance,non_moral,True,True,-5.127501,152.000000,two-sided,0.000001,0.000005,bonf,1.355e+04,-0.586239
3,cond,-,chance,personal_moral,True,True,4.354625,152.000000,two-sided,0.000024,0.000146,bonf,586.852,0.497875
4,cond,-,impersonal_moral,non_moral,True,True,-1.931563,152.000000,two-sided,0.055274,0.331644,bonf,0.55,-0.211932
5,cond,-,impersonal_moral,personal_moral,True,True,4.689400,152.000000,two-sided,0.000006,0.000036,bonf,2180.903,0.487864
6,cond,-,non_moral,personal_moral,True,True,6.865413,152.000000,two-sided,0.000000,0.000000,bonf,5.329e+07,0.758009
7,engine * cond,dv2,chance,impersonal_moral,True,True,-0.074930,76.000000,two-sided,0.940468,1.000000,bonf,0.126,-0.012076
8,engine * cond,dv2,chance,non_moral,True,True,-4.642073,76.000000,two-sided,0.000014,0.000170,bonf,1235.799,-0.748138
9,engine * cond,dv2,chance,personal_moral,True,True,1.924264,76.000000,two-sided,0.058065,0.696781,bonf,0.721,0.310123


In [8]:
# add a chance condition
avg = bias[bias.engine.isin(['dv2', 'dv3', 'dv2_humanized'])].groupby(['engine', 'prolific_id', 'cond'], as_index=False).mean(numeric_only=True)

to_add = []
for engine in avg.engine.unique():
    for id in avg[avg.engine==engine].prolific_id.unique():
        for cond in ['chance']:
            to_add.append(
                {
                    'engine': engine,
                    'prolific_id': id,
                    'cond': cond,
                    'bias_answer': 0,
                    'bias_source': 0
                }
            )
to_add = pd.DataFrame(to_add)

avg = pd.concat([avg, to_add])

pg.pairwise_tests(dv='bias_answer', within='cond', between="engine",
                   subject='prolific_id', data=avg, padjust='bonf', within_first=True, effsize='cohen').round(6)

,Contrast,cond,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,cond,-,chance,impersonal_moral,True,True,-3.254067,216.000000,two-sided,0.001320,0.007922,bonf,12.489,-0.312400
1,cond,-,chance,non_moral,True,True,-6.420718,216.000000,two-sided,0.000000,0.000000,bonf,9.446e+06,-0.616409
2,cond,-,chance,personal_moral,True,True,-4.802110,216.000000,two-sided,0.000003,0.000018,bonf,3794.317,-0.461017
3,cond,-,impersonal_moral,non_moral,True,True,-2.449919,216.000000,two-sided,0.015085,0.090507,bonf,1.41,-0.203201
4,cond,-,impersonal_moral,personal_moral,True,True,-1.139556,216.000000,two-sided,0.255734,1.000000,bonf,0.144,-0.087762
5,cond,-,non_moral,personal_moral,True,True,1.516224,216.000000,two-sided,0.130925,0.785549,bonf,0.235,0.120338
6,engine,-,dv2,dv2_humanized,False,True,1.123211,142.021586,two-sided,0.263243,0.789730,bonf,0.316,0.185453
7,engine,-,dv2,dv3,False,True,1.918077,136.862914,two-sided,0.057184,0.171552,bonf,0.955,0.317591
8,engine,-,dv2_humanized,dv3,False,True,0.531282,127.314338,two-sided,0.596149,1.000000,bonf,0.204,0.088189
9,cond * engine,chance,dv2,dv2_humanized,False,True,NaN,1.000000,two-sided,NaN,NaN,bonf,nan,NaN


## 4.4 Corpus 3

In [13]:
avg = df[df.engine.isin(['dv2_humanized'])].groupby(['engine', 'prolific_id'], as_index=False).mean(numeric_only=True)

pg.ttest(avg.correct, 0.5, correction='bonf').round(20)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,8.247513,76,two-sided,3.695383e-12,"[0.56, 0.59]",0.939891,2.304e+09,1.0


In [14]:
# add a chance condition
avg = bias2[bias2.engine.isin(['dv2_humanized'])].groupby(['engine', 'prolific_id'], as_index=False).mean(numeric_only=True)

# pg.pairwise_tests(dv='bias_answer', between='engine', within="cond", data=avg, padjust='bonf', within_first=True, effsize='cohen').round(6)
pg.ttest(avg.bias_answer, 0, correction='bonf').round(20)



,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,3.571878,75,two-sided,0.000623,"[0.04, 0.15]",0.409723,37.828,0.941304


## 4.5 Linguistic

In [15]:
def first_person(x):
    for w in x.split():
        if w.lower() in ['i', 'me', 'my', 'mine', 'myself']:
            return 1
    return 0

df['first_person'] = df['a1'].apply(lambda x: first_person(x))
df['len'] = df['a1'].apply(lambda x: len(x.split()))

# remove Human source from DV3, DV2 humanized
df2 = df[~((df.source=='Human') & (df.engine=='dv3'))]
df2 = df2[~((df2.source=='Human') & (df2.engine=='dv2_humanized'))]
df2 = df2[~((df2.source=='Human') & (df2.engine=='dv2'))]
# now add a human engine with all rows from df.source=='Human' and engine='dv2'
new_df = df[(df.source=='Human') & (df.engine=='dv2')]
new_df['engine'] = 'human'

dfl = pd.concat([df2, new_df])

In [16]:
pg.pairwise_ttests(data=dfl, dv='len', between='engine', subject='prolific_id', padjust='bonf', effsize='cohen').round(6)


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,engine,dv2,dv2_humanized,False,True,43.743982,1890.488266,two-sided,0.000000,0.000000,bonf,inf,1.471506
1,engine,dv2,dv3,False,True,-16.536675,3276.775128,two-sided,0.000000,0.000000,bonf,3.269e+55,-0.560748
2,engine,dv2,human,False,True,42.129296,2079.836108,two-sided,0.000000,0.000000,bonf,inf,1.418438
3,engine,dv2_humanized,dv3,False,True,-81.429009,1842.338194,two-sided,0.000000,0.000000,bonf,inf,-2.869404
4,engine,dv2_humanized,human,False,True,-1.511467,2982.244339,two-sided,0.130776,0.784653,bonf,0.118,-0.050744
5,engine,dv3,human,False,True,77.578323,2128.509497,two-sided,0.000000,0.000000,bonf,inf,2.721378


In [17]:
pg.pairwise_ttests(data=dfl, dv='first_person', between='engine', subject='prolific_id', padjust='bonf', effsize='cohen').round(6)

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,engine,dv2,dv2_humanized,False,True,8.194804,3300.432049,two-sided,0.000000,0.000000,bonf,9.273e+12,0.275717
1,engine,dv2,dv3,False,True,2.758238,3417.699738,two-sided,0.005842,0.035054,bonf,1.694,0.094144
2,engine,dv2,human,False,True,-11.620929,3431.778406,two-sided,0.000000,0.000000,bonf,1.781e+27,-0.390363
3,engine,dv2_humanized,dv3,False,True,-5.296283,3204.334134,two-sided,0.000000,0.000001,bonf,4.231e+04,-0.182414
4,engine,dv2_humanized,human,False,True,-20.018886,3004.665300,two-sided,0.000000,0.000000,bonf,5.774e+80,-0.672106
5,engine,dv3,human,False,True,-14.309629,3312.183970,two-sided,0.000000,0.000000,bonf,4.125e+41,-0.484707


In [18]:
pg.pairwise_ttests(data=dfl, dv='errors', between='engine', subject='prolific_id', padjust='bonf', effsize='cohen').round(6)

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,engine,dv2,dv2_humanized,False,True,-15.515805,2078.363789,two-sided,0.000000,0.0,bonf,9.465e+48,-0.522198
1,engine,dv2,dv3,False,True,-0.829691,3405.481986,two-sided,0.406772,1.0,bonf,0.054,-0.028385
2,engine,dv2,human,False,True,-16.873642,2172.633079,two-sided,0.000000,0.0,bonf,7.013e+57,-0.566023
3,engine,dv2_humanized,dv3,False,True,15.140079,2094.335301,two-sided,0.000000,0.0,bonf,3.899e+46,0.503986
4,engine,dv2_humanized,human,False,True,0.304718,3486.775335,two-sided,0.760599,1.0,bonf,0.04,0.010245
5,engine,dv3,human,False,True,-16.446289,2192.167438,two-sided,0.000000,0.0,bonf,8.34e+54,-0.546389


## 4.6 Predictive modeling

In [19]:

# plt.figure(figsize=(15, 5))
df['len<median'] = (df.len < df.len.median()).astype(int)
df['errors>0'] = (df.errors > 0).astype(int)
df['first_person'] = df.first_person.astype(int)
df['agree'] = np.mean([df.agree1, df.agree2], axis=0)

# avg = df.groupby(['engine', 'prolific_id'], as_index=False).mean(numeric_only=True)

pg.pairwise_tests(dv='p_human', within="len<median", between="engine", subject='prolific_id', data=df, padjust='bonf', effsize='cohen', within_first=False).round(6)


,Contrast,engine,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,engine,-,dv2,dv2_humanized,False,True,-0.464529,152.000000,two-sided,0.642933,1.0,bonf,0.192,-0.074866
1,engine,-,dv2,dv3,False,True,-0.748093,150.941581,two-sided,0.455568,1.0,bonf,0.225,-0.120967
2,engine,-,dv2_humanized,dv3,False,True,-0.293482,150.779623,two-sided,0.769558,1.0,bonf,0.181,-0.047462
3,len<median,-,0,1,True,True,-10.357190,229.000000,two-sided,0.000000,NaN,NaN,5.962e+17,-0.957315
4,engine * len<median,dv2,0,1,True,True,-8.000124,76.000000,two-sided,0.000000,0.0,bonf,8.05e+08,-1.216231
5,engine * len<median,dv2_humanized,0,1,True,True,0.133735,76.000000,two-sided,0.893966,1.0,bonf,0.127,0.016416
6,engine * len<median,dv3,0,1,True,True,-10.925474,75.000000,two-sided,0.000000,0.0,bonf,1.652e+14,-1.727053


In [20]:
pg.pairwise_tests(dv='p_human', within="errors>0", between="engine", subject='prolific_id', data=df, padjust='bonf', effsize='cohen', within_first=False).round(6)

,Contrast,engine,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,engine,-,dv2,dv2_humanized,False,True,-0.260159,145.409490,two-sided,0.795109,1.000000,bonf,0.18,-0.042117
1,engine,-,dv2,dv3,False,True,-1.580138,150.000000,two-sided,0.116182,0.348545,bonf,0.548,-0.256332
2,engine,-,dv2_humanized,dv3,False,True,-1.468323,149.957049,two-sided,0.144111,0.432333,bonf,0.468,-0.237268
3,errors>0,-,0,1,True,True,-9.123446,228.000000,two-sided,0.000000,NaN,NaN,1.294e+14,-0.749754
4,engine * errors>0,dv2,0,1,True,True,-4.016278,75.000000,two-sided,0.000139,0.000417,bonf,149.147,-0.582722
5,engine * errors>0,dv2_humanized,0,1,True,True,-3.097572,76.000000,two-sided,0.002734,0.008201,bonf,9.946,-0.411281
6,engine * errors>0,dv3,0,1,True,True,-10.330071,75.000000,two-sided,0.000000,0.000000,bonf,1.391e+13,-1.414647


In [21]:
pg.pairwise_tests(dv='p_human', within="first_person", between="engine", subject='prolific_id', data=df, padjust='bonf', effsize='cohen', within_first=False).round(6)

,Contrast,engine,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,engine,-,dv2,dv2_humanized,False,True,-1.835823,152.000000,two-sided,0.068338,0.205014,bonf,0.812,-0.295870
1,engine,-,dv2,dv3,False,True,-0.973678,150.918451,two-sided,0.331775,0.995324,bonf,0.269,-0.157400
2,engine,-,dv2_humanized,dv3,False,True,0.885056,150.969801,two-sided,0.377534,1.000000,bonf,0.25,0.143082
3,first_person,-,0,1,True,True,-16.178720,229.000000,two-sided,0.000000,NaN,NaN,2.671e+36,-1.248532
4,engine * first_person,dv2,0,1,True,True,-7.312867,76.000000,two-sided,0.000000,0.000000,bonf,4.423e+07,-1.044931
5,engine * first_person,dv2_humanized,0,1,True,True,-8.728075,76.000000,two-sided,0.000000,0.000000,bonf,1.786e+10,-1.092432
6,engine * first_person,dv3,0,1,True,True,-13.282227,75.000000,two-sided,0.000000,0.000000,bonf,2.143e+18,-1.675419
